In [1]:
import openai

In [2]:
from openai import OpenAI

In [9]:
client = OpenAI(api_key="sk-OPEN_AI_API_KEY")

In [10]:
response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This image showcases a serene landscape with a wooden boardwalk stretching through a lush green meadow under a clear sky dotted with a few clouds. The boardwalk invites viewers to venture through the tall grasses and wild plants that thrive on either side. The whole scene is lit by what seems to be late afternoon sunlight, enhancing the vivid greens and blues of the natural elements. This setting is typical of nature reserves or rural pathways intended to minimize human impact on the environment while allowing for enjoyment and exploration of natural settings.', role='assistant', function_call=None, tool_calls=None))
